In [1]:
import utils

	
 !"#%&'()*+,-./0123456789:<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}ĆćčđŠšžНУабвгдезиклмнопрстухчшђјњћ…⁠✋❤﻿🏻🏼🏽👋👌👍👽💪😀😂😃😍😟🙂🙌🙏🤑🤘🤙🤝🤢🤣🤮🥰🥷🥹🧠🫂🫶
159
Step 0: train loss 5.5599, val loss 5.5486
Step 1000: train loss 4.2775, val loss 4.2967
Step 2000: train loss 3.3423, val loss 3.4014
Step 3000: train loss 2.7434, val loss 2.8484
Step 4000: train loss 2.4238, val loss 2.5454
Step 5000: train loss 2.2727, val loss 2.4024
Step 6000: train loss 2.1916, val loss 2.3212
Step 7000: train loss 2.1470, val loss 2.2895
Step 8000: train loss 2.1150, val loss 2.2614
Step 9000: train loss 2.1398, val loss 2.2525


In [1]:
utils.intuition_test('trivan')

NameError: name 'utils' is not defined

In [39]:
import torch
from torch.nn import functional as F
torch.manual_seed(1111)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

# v1
xbow = torch.zeros((B, T, C))
for b in range (B):
    for t in range (T):
        xprev = x[b, :t + 1]
        xbow[b, t] = torch.mean(xprev, 0)

# v2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)

# v3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

torch.allclose(xbow2, xbow3)

True